# 利用`__slots__`限制对实例动态添加的属性和方法

Python是一门动态语言，即类的属性可以动态添加。在定义一个类的时候，可以定义一些属性和方法；在对类进行实例化后，也可以为这些实例添加属性和方法，但对实例添加的属性和方法只对这个实例有效，其他实例则无法访问。

In [25]:
import traceback                # 处理异常的时候，可以使用traceback打印相关信息
from types import MethodType

class MyClass(object):
    pass

def set_name(self, name):
    self.name = name

# 实例化
cls1 = MyClass()
# 动态添加属性
cls1.name = 'Tom'
print(cls1.name)
# 动态添加方法
cls1.set_name = MethodType(set_name, cls1)
cls1.set_name('Jerry')
print(cls1.name)

Tom
Jerry


In [26]:
# 新的实例无法访问name属性和set_name()方法
cls2 = MyClass()
cls2.name

AttributeError: 'MyClass' object has no attribute 'name'

In [27]:
cls2.set_name('Sean')

AttributeError: 'MyClass' object has no attribute 'set_name'

如果在定义一个新的类的时候，可以利用`__slots__`对后续通过实例动态添加的属性和方法进行限制。实际应用中可以防止用户对程序添加意外的不需要的属性或方法。

In [28]:
import traceback                # 处理异常的时候，可以使用traceback打印相关信息
from types import MethodType

class MyClass(object):
# 只能动态添加名为name或set_name的属性或方法
    __slots__ = ['name', 'set_name']

def set_name(self, name):
    self.name = name

# 实例化
cls1 = MyClass()
# 动态添加属性
cls1.name = 'Tom'
print(cls1.name)
# 动态添加方法
cls1.set_name = MethodType(set_name, cls1)
cls1.set_name('Jerry')
print(cls1.name)

# 动态添加age属性就会报错
try:
    cls1.age = 30
except AttributeError:
    traceback.print_exc()

Tom
Jerry


Traceback (most recent call last):
  File "<ipython-input-28-ad86a9a831f1>", line 23, in <module>
    cls1.age = 30
AttributeError: 'MyClass' object has no attribute 'age'


`__slots__`只对当前类的实例起作用，而对继承的子类的实例是不起作用的。

In [29]:
# MyClass类中的__slots__对其子类ExtMyClass是不起作用的。

class ExtMyClass(MyClass):
    pass

ext_cls = ExtMyClass()
ext_cls.age = 30
print(ext_cls.age)

30


# 利用`type()`函数动态创建一个类

In [3]:
def init(self, name):
    self.name = name
    
def say_hello(self):
    print('Hello, %s!' % self.name)
    
Hello = type('Hello', (object,), dict(__init__=init, hello=say_hello))
# 第一个参数：类的名字
# 第二个参数：继承自哪些类（元组）
# 第三个参数：属性及方法（字典）

h = Hello('Tom')
h.hello()

Hello, Tom!


# 利用元类控制类的构造方式

通过修改元类中`__new__()`函数中的`attrs`参数来控制类中的属性和方法。 元类都是继承自`type`的。

以下代码构造一个继承自`list`的类`MyList`，并通过元类为其中添加一个`add()`方法，该方法实际上等同于`append()`。

In [9]:
# 方法1

# 定义元类
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        # print(cls)
        # print(name)
        # print(bases)
        # print(attrs)
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

class MyList(list, metaclass=ListMetaclass):
    pass

my_li = MyList()
my_li.add(1)
my_li.add(2)
my_li.add(3)
print(my_li)

[1, 2, 3]


In [10]:
# 方法2

# 定义add()方法
def add(self, value):
    return self.append(value)

# 定义元类
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        # print(cls)
        # print(name)
        # print(bases)
        # print(attrs)
        attrs['add'] = add
        return type.__new__(cls, name, bases, attrs)

class MyList(list, metaclass=ListMetaclass):
    pass

my_li = MyList()
my_li.add(1)
my_li.add(2)
my_li.add(3)
print(my_li)

[1, 2, 3]


In [11]:
# 也可以添加属性

# 定义add()方法
def add(self, value):
    return self.append(value)

# 定义元类
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        # print(cls)
        # print(name)
        # print(bases)
        # print(attrs)
        attrs['add'] = add
        attrs['name'] = 'Tom'
        return type.__new__(cls, name, bases, attrs)

class MyList(list, metaclass=ListMetaclass):
    pass

my_li = MyList()
my_li.add(1)
my_li.add(2)
my_li.add(3)
print(my_li)
print(my_li.name)

[1, 2, 3]
Tom


# 利用元类实现一个简单的数据库

利用元类实现一个简单的数据库，并实现`save()`函数。

In [24]:
class Field(object):
    def __init__(self, name, col_type):
        self.name = name
        self.col_type = col_type
        
class IntegerField(Field):
    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'integer')
        
class StringField(Field):
    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(1024)')
        
        
class ModelMetaclass(type):
    def __new__(cls, name, bases, attrs):
        if name == 'Model':
            return type.__new__(cls, name, bases, attrs)
        print('Model name: %s' % name)
        mappings = {}
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Field name: %s' % k)
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings
        attrs['__table__'] = name
        return type.__new__(cls, name, bases, attrs)
    

class Model(dict, metaclass=ModelMetaclass):
    def __init__(self, **kvs):
        super(Model, self).__init__(**kvs)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError("'Model' object has no attribute '%s'." % key)
            
    def __setattr__(self, key, value):
        self[key] = value
        
    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s(%s) values(%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('sql: ', sql)
        print('args: ', args)
    


class User(Model):
    Id = IntegerField('id')
    name = StringField('name')
    
u = User()
u.Id = 100
u.name = 'Tom'
u.save()

Model name: User
Field name: Id
Field name: name
Id
id
name
name
sql:  insert into User(id,name) values(?,?)
args:  [100, 'Tom']
